# Import Libraries

In [1]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import swifter

/home/rohit/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rohit/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rohit/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rohit/.local/lib/python3.7/site-packages/tensorboard/compat/tensorf

# Preprocessing Function

In [2]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)

def remove_non_nouns(text):
    tokens = nltk.word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    filter_tokens = [t[0] for t in tags if t[1] == "NN" or t[1] == "VB"]
    return ' '.join(filter_tokens)


def remove_custom_stopwords(s):
    my_stop_words = STOPWORDS.union(set(['time', 'year', 'number', 'today', 'week', 'month', 'night', 'world', 'home',
                                         'place', 'yesterday', 'life', 'wife']))
    return " ".join(w for w in s.split() if w not in my_stop_words)


def preprocess_text_for_lda(text):
    LDA_FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   remove_custom_stopwords, strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, LDA_FILTERS)

def filter_multiple_topics(topic):
    if '|' in topic:
        return topic.split('|')[0].strip()
    return topic

def preprocess(topic):
    t = filter_multiple_topics(topic)
    if t == 'admin':
        t = 'Others'
        
    return topics_name_to_index_map[t]

# Predict Function

In [3]:
def predict(X, topn=3):
    preprocessed_X = preprocess_text_for_lda(X)
    vec_X = doc2vec_model.infer_vector(preprocessed_X)
    vec_X = vec_X.reshape(1, vec_X.shape[0])
    pred = classifier.predict(vec_X)
    pred_i = [(topics_index_to_name_map[i], p) for i,p in enumerate(pred[0])]
    pred_sorted = sorted(pred_i, key=lambda x: x[1], reverse=True)
    return pred_sorted[:topn]

# Network Architecture

In [4]:
def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(21, activation='softmax')
    ])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0, name='categorical_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

# Load Saved Models

In [5]:
doc2vec_model = Doc2Vec.load('doc2vec')
classifier = build_network()
classifier.load_weights('./topics_classifier_2')

# Test for parliament debates

In [115]:
df = pd.read_csv('./data/2014_debate.csv')
df = df.drop(df[df.topic == 'admin'].index)
df = df.drop(df[df.transcript.str.split().map(len) < 10].index).reset_index()
df['topic'] = df.apply(lambda row: preprocess(row['topic']), axis=1)

In [116]:
df

,index,topic,transcript
0,0,8,1. What assessment he has made of the perform...
1,1,8,2. How many applications his Department has r...
2,2,8,3. What steps his Department is taking to imp...
3,3,8,4. What systems his Department has in place f...
4,4,8,5. What progress he has made on encouraging t...
...,...,...,...
1839,1892,10,11. What recent assessment he has made of tre...
1840,1893,10,16. What estimate he has made of how much sub...
1841,1894,4,(Urgent Question): To ask the Secretary of Sta...
1842,1895,13,(Urgent Question): To ask the Minister to make...


In [119]:
X = df['transcript'].values
Y = df['topic'].values
i = 45
print(predict(X[i]))
print('true: ', Y[i], topics_index_to_name_map[Y[i]])

[('Agriculture, animals, food and rural affairs', 0.848889), ('Energy and environment', 0.050586324), ('Parliament, government and politics', 0.047453415)]
true:  0 Agriculture, animals, food and rural affairs


In [120]:
X[i]

'2.  What assessment he has made of the scope for cutting red tape in the farming industry. [901855]  We are committed to freeing farmers from red tape to help them to seize economic opportunities. We are reducing paperwork burdens and making guidance clearer and simpler. Farmers who play by the rules now receive fewer inspections. For example, 740 members of the Environment Agency’s pig and poultry scheme are inspected once every three years, rather than annually. I expect to make an announcement shortly on further opportunities for cutting red tape as a result of the agriculture red tape challenge.  I thank the Minister for that answer, but for many farmers in my constituency overly complex livestock identification and movement controls remain a burden on their businesses. What plans does the Minister have to simplify this regime?  My hon. Friend makes a good point. Considerable progress has already been made on livestock identification and the complex rules governing animal movement

In [121]:
df['topic'] = df.apply(lambda row: topics_index_to_name_map[row['topic']], axis=1)

In [122]:
df['predicted_topic'] = df.swifter.apply(lambda row: predict(row['transcript']), axis=1)

In [123]:
df.to_csv('./doc2vec_debate_pred.csv')

In [124]:
df

,index,topic,transcript,predicted_topic
0,0,Education,1. What assessment he has made of the perform...,"[(Education, 0.9740481), (Business, industry a..."
1,1,Education,2. How many applications his Department has r...,"[(Education, 0.98344123), (Business, industry ..."
2,2,Education,3. What steps his Department is taking to imp...,"[(Education, 0.44933805), (Business, industry ..."
3,3,Education,4. What systems his Department has in place f...,"[(Education, 0.7928537), (Crime, civil law, ju..."
4,4,Education,5. What progress he has made on encouraging t...,"[(Education, 0.8934695), (Business, industry a..."
...,...,...,...,...
1839,1892,Energy and environment,11. What recent assessment he has made of tre...,"[(Energy and environment, 0.9285043), (Economy..."
1840,1893,Energy and environment,16. What estimate he has made of how much sub...,"[(Energy and environment, 0.6575643), (Economy..."
1841,1894,"Crime, civil law, justice and rights",(Urgent Question): To ask the Secretary of Sta...,"[(Health services and medicine, 0.8022428), (T..."
1842,1895,Housing and planning,(Urgent Question): To ask the Minister to make...,"[(International affairs, 0.37547308), (Economy..."


# Test for Nexis news data

In [12]:
df = pd.read_csv('./data/news.csv', encoding='utf8')
df = df.dropna(subset=['transcript'])
df = df.drop(['Unnamed: 0'], axis=1)

In [13]:
df

147284

In [ ]:
df['predicted_topic'] = df.swifter.apply(lambda row: predict(row['transcript']), axis=1)

In [ ]:
df

In [ ]:
df.to_csv('./doc2vec_news_pred.csv')

# Test for BBC transcripts

In [6]:
df = pd.read_csv('./data/News jan-2014 175.csv')

In [9]:
df = df.drop(['Unnamed: 0', 'Program Name', 'Has Transcript', 'Unavailable link', 'Unavailable reason'], axis=1)

,Source,Date,Time,Duration,Transcript
0,ITV London,4-jan-2014,16:30,15 mins,"may am Wdievasiltailmdfm Mw mms say they are ""..."
1,ITV London,1-jan-2014,22:30,20 mins,Welcome to the UK. All Romanians and Bulgarian...
2,ITV London,2-jan-2014,22:00,35 mins,Get up. Come on. What have you done here? Oh.....
3,ITV London,6-jan-2014,22:00,35 mins,"New Year and a new split in the Coalition, as ..."
4,ITV London,3-jan-2014,13:30,30 mins,"W9 fired from that embassy, we were actually p..."
5,ITV London,3-jan-2014,22:00,35 mins,(VOICE DROWNED OUT BY APPLAUSE) (CHEERING AND ...
6,ITV London,7-jan-2014,13:30,30 mins,Wha The child bomber in Afganistan. The 10-yea...
7,ITV London,7-jan-2014,22:00,35 mins,hopefully we all have a better chance of spott...
8,ITV London,8-jan-2014,13:30,30 mins,sad time. It’s a desperately sad time. Also th...
9,ITV London,8-jan-2014,18:30,30 mins,rain across the country. The rain clears off f...


In [11]:
df['predicted_topic'] = df.swifter.apply(lambda row: predict(row['Transcript']), axis=1)

In [12]:
df

,Source,Date,Time,Duration,Transcript,predicted_topic
0,ITV London,4-jan-2014,16:30,15 mins,"may am Wdievasiltailmdfm Mw mms say they are ""...","[(Parliament, government and politics, 0.31012..."
1,ITV London,1-jan-2014,22:30,20 mins,Welcome to the UK. All Romanians and Bulgarian...,"[(Parliament, government and politics, 0.31876..."
2,ITV London,2-jan-2014,22:00,35 mins,Get up. Come on. What have you done here? Oh.....,"[(Others, 0.25380623), (Transport, 0.17195277)..."
3,ITV London,6-jan-2014,22:00,35 mins,"New Year and a new split in the Coalition, as ...","[(Others, 0.3411245), (Crime, civil law, justi..."
4,ITV London,3-jan-2014,13:30,30 mins,"W9 fired from that embassy, we were actually p...","[(Others, 0.39453667), (Energy and environment..."
5,ITV London,3-jan-2014,22:00,35 mins,(VOICE DROWNED OUT BY APPLAUSE) (CHEERING AND ...,"[(Energy and environment, 0.2139581), (Interna..."
6,ITV London,7-jan-2014,13:30,30 mins,Wha The child bomber in Afganistan. The 10-yea...,"[(Others, 0.3256316), (Parliament, government ..."
7,ITV London,7-jan-2014,22:00,35 mins,hopefully we all have a better chance of spott...,"[(Others, 0.43381527), (Communities and famili..."
8,ITV London,8-jan-2014,13:30,30 mins,sad time. It’s a desperately sad time. Also th...,"[(Parliament, government and politics, 0.25211..."
9,ITV London,8-jan-2014,18:30,30 mins,rain across the country. The rain clears off f...,"[(Others, 0.65529424), (Parliament, government..."


In [13]:
df.to_csv('./doc2vec_bbc_pred.csv')